In [ ]:
# In order to train model, first the training data needs to be made

# Therefore data before 2000 will be made into sentence and context, 
#but blame will only be assigned for the sentence in question

In [1]:
import spacy
import pandas as pd

In [ ]:
#junk
#df['date'] = pd.to_datetime(df['date'])
#df.to_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/Corp_Folketing_V2.csv")

'''
from datetime import date
annotation_data = df[(df['date'].dt.date <= date(2000,1,1))]
annotation_data.to_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/annotation_data.csv")
'''

In [2]:
#load in data

df = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/Corp_Folketing_V2.csv")

df.head()


,Unnamed: 0,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
0,0,1997-10-07,Dagsorden,1,Gert Petersen,NaN,NaN,True,191,Mødet er åbnet. I henhold til grundloven er Fo...,DK-Folketing,DNK
1,1,1997-10-07,Dagsorden,2,Formanden,NaN,NaN,True,182,"Jeg vil gerne takke Tinget for den tillid, man...",DK-Folketing,DNK
2,2,1997-10-07,Statsministerens redegørelse i henhold til gru...,3,Poul Nyrup Rasmussen,S,379.0,False,18662,For 25 år siden sagde et flertal i befolkninge...,DK-Folketing,DNK
3,3,1997-10-09,1) Indstilling fra Udvalget til Valgs Prøvelse.,2,Formanden,NaN,NaN,True,47,Fra Udvalget til Valgs Prøvelse har jeg modtag...,DK-Folketing,DNK
4,4,1997-10-09,2) Forhandling om redegørelse nr. R 1.,3,Torben Lund,S,379.0,False,2865,Vi står over for en meget afgørende folketings...,DK-Folketing,DNK


In [12]:
import re

def simplify_agenda(text):
    text = text.lower()  # make it case-insensitive
    text = text.strip()
    
    # Match anything containing "behandling af beslutningsforslag"
    if re.search(r'behandling af beslutningsforslag', text):
        return 'behandling af beslutningsforslag'
    
    elif re.search(r'forhandling', text):
        return 'forhandling'

    elif re.search(r'spørgsmål om fremme af forespørgsel', text):
        return 'spørgsmål om fremme af forespørgsel'

    elif re.search(r'spørgsmål til ministrene', text):
        return 'spørgsmål til ministrene'

    elif re.search(r'behandling af lovforslag', text):
        return 'behandling af lovforslag'

    elif re.search(r'fortsættelse af forespørgsel', text):
        return 'fortsættelse af forespørgsel'

    elif re.search(r'spørgsmål om meddelelse af orlov til', text):
        return 'spørgsmål om meddelelse af orlov til'


    elif re.search(r'indstilling fra udvalget til valgs prøvelse', text):
        return 'indstilling fra udvalget til valgs prøvelse'

    elif re.search(r'til ministeren', text):
        return 'til ministeren'

    
    
    else:
        return text
    
# Apply to your dataframe
df['agenda_simplified'] = df['agenda'].apply(simplify_agenda)


In [13]:
#df['agenda_simplified'].value_counts()
df['agenda_simplified'].unique()

array(['dagsorden',
       'statsministerens redegørelse i henhold til grundlovens § 38 (mundtlig del)',
       'indstilling fra udvalget til valgs prøvelse', ...,
       'forespørgsel nr. f 3:  forespørgsel til udlændinge- og integrationsministeren:  kan ministeren oplyse, om regeringen ønsker at forhindre udenlandske pengeoverførsler til byggeri af moskeer og til muslimske kulturelle, ideologiske og religiøse formål i danmark, og i givet fald, hvordan?  af martin henriksen (df), marie krarup (df) og christian langballe (df).  (anmeldelse 03.10.2018. fremme 09.10.2018).  kl. 14:29',
       'forespørgsel nr. f 16:  forespørgsel til beskæftigelsesministeren:  hvad kan ministeren oplyse om sin holdning til ekspertudvalget om udredning af arbejdsmiljøindsatsens anbefalinger i rapporten »et nyt og forbedret arbejdsmiljø – overvejelser og anbefalinger«, som blev offentliggjort den 27. september 2018, og hvad vil ministeren konkret gøre for at styrke det forebyggende arbejde og nedbringe ant

In [11]:
import random

# Example: annotation_data is your original DataFrame
paragraphs = annotation_data['text'].tolist()
original_indices = list(annotation_data.index)  # keep original index for para_id

# Shuffle paragraphs (zip together with original index)
zipped = list(zip(original_indices, paragraphs))
random.shuffle(zipped)

# Unzip
shuffled_indices, shuffled_paragraphs = zip(*zipped)


In [ ]:


# Load Danish pipeline
nlp = spacy.load("da_core_news_sm")

def split_paragraph(paragraph: str):
    doc = nlp(paragraph)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences


In [7]:
def build_samples(paragraph):
    sentences = split_paragraph(paragraph)
    samples = []
    
    for i, sent in enumerate(sentences):
        prev_sent = sentences[i-1] if i > 0 else ""
        next_sent = sentences[i+1] if i < len(sentences)-1 else ""
        
        #text = prev_sent + " [SEP] " + sent + " [SEP] " + next_sent

        sent_id = i

        # Three-part input with [SEP] handled by tokenizer later
        samples.append({
            "prev": prev_sent,
            "target": sent,
            "next": next_sent
        })
        
    return samples


In [8]:
def annotate_sentence(sentence, para_id, sent_id):
    """
    Annotate a sentence interactively.
    
    Returns:
        label (int) or None if escaped
    """
    print(f"\nPara {para_id}, Sent {sent_id}:")
    print(f"\"{sentence}\"")
    
    while True:
        user_input = input("Label this sentence (0 = no blame, 1 = blame, 'q' to quit): ").strip()
        if user_input.lower() == 'q':
            return None  # escape
        elif user_input in ['0', '1']:
            return int(user_input)
        else:
            print("Invalid input. Please enter 0, 1, or 'q'.")


In [ ]:
labelled_sentences = []

for para_id, para in zip(shuffled_indices, shuffled_paragraphs):
    samples = build_samples(para)  # build prev/target/next windows

    for sent_idx, s in enumerate(samples):
        # annotate interactively
        label = annotate_sentence(s['target'], para_id, sent_idx)
        if label is None:  # user pressed 'q'
            print("Annotation interrupted. Saving progress...")
            break
        
        labelled_sentences.append({
            "para_id": para_id,   # original index
            "sent_id": sent_idx,
            "prev": s['prev'],
            "target": s['target'],
            "next": s['next'],
            "label": label
        })
    else:
        continue  # only if inner loop wasn't broken
    break  # breaks outer loop if user quits


In [10]:
#save data
data = pd.DataFrame(labelled_sentences)
data.to_csv("annotated_sentences.csv", index=False)

In [ ]:
#later when tokenizing:
#HuggingFace will automatically insert [SEP] between these segments.
#For BERT-like models, it also uses segment embeddings to distinguish the three parts.

'''
def tokenize(batch):
    return tokenizer(
        batch["prev"], 
        batch["target"], 
        batch["next"], 
        truncation=True, 
        padding="max_length"
    )
'''


In [ ]:
pip install transformers datasets evaluate

import pandas as pd
from datasets import Dataset

df = pd.read_csv("annotated_sentences.csv")

# Keep only the necessary columns
df = df[['prev', 'target', 'next', 'label']]

# Convert to HuggingFace Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)


from transformers import AutoTokenizer

model_name = "bert-base-multilingual-cased"  # works for Danish
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(
        example['prev'],
        example['target'],
        example['next'],
        truncation=True,
        padding='max_length',
        max_length=256 #OBS
    )

#tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=False)


#initialize model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

from sklearn.model_selection import train_test_split


# Split tokenized dataset
splits = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = splits['train']
test_dataset = splits['test']


#define metrics / Two options have been listed below. The second i probably preferred as ittakes the amount of 0 to 1' into account
# First option
import evaluate
import numpy as np

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

#second option
import torch
from collections import Counter

labels = df['label'].tolist()
class_counts = Counter(labels)
total = sum(class_counts.values())

# Higher weight = more emphasis
weights = [total/class_counts[0], total/class_counts[1]]
class_weights = torch.tensor(weights, dtype=torch.float)

#define custom trainer that uses weigted loss
from transformers import Trainer
import torch.nn as nn

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # Define weighted loss
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

# And implement the second option in training
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()



#set up trainer
from transformers import TrainingArguments, Trainer

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./blame_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

#train

trainer.train()







